<a href="https://colab.research.google.com/github/hienvm/DeepwalkDemo/blob/main/deepwalk_blogcatalog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'blogcatalog:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5085144%2F8517227%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240527%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240527T180050Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D72f11efe7ffb8655c2897ecf4aeb8eb7d9f8abfb27e96fababe385fdd1cc0e3c62ea52ec338bf4b826f00935ee3d92c9f34ec08720639f9ae76ff98b2348566c27afef12b84435fe0bebc77b95fda8e731ccf0b29d0e32bbd9e66b6a4c113c9a39ec5936df1665098b6988ec7900bdffdbf881c2dd388aa9d002cfdad4a8d0c7a31ef02595e1019e3c6982298359fd135ef0d8ba3b7093cc673815a26ac82043935c29bc3c8ef1a0ddb79d5832fc94fb94ef3577114bcab65c48ed4c32f2b2102989f5992ef325030f9eb8bc736e1916dde54120e35dfb1e228c3049bdb1ea30e4fb3cb0cdac8f6ca9e3165574bd313f9a0d07f47ef67ac8a9105f0b582e43f7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Deepwalk Implementation cho BlogCatalog


In [ ]:
import torch
import matplotlib.pyplot as plt
import networkx as nx
from torch import tensor, Tensor
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
import numpy.random as random
import torch.multiprocessing as mp

Các Hyperparameter


In [ ]:
EMBEDDING_SIZE = 64
WALK_LENGTH = 20
WALKS_PER_VERTEX = 10
WINDOW_SIZE = 5
START_LEARNING_RATE = 0.025
END_LEARNING_RATE = 0.005
# Tiền xử lý 1D embedding
PREPROCESS_WALKS_PER_VERTEX = 10
# Dùng cho TSNE
PERPLEXITY = 30

device và worker_threads


In [ ]:
WORKER_THREADS = torch.cuda.device_count(
) if torch.cuda.is_available() else mp.cpu_count()
WORKER_THREADS = 1
CHUNK_SIZE = 4

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(WORKER_THREADS)

Load dataset


In [ ]:
df = pd.read_csv("/kaggle/input/blogcatalog/edges.csv", header=None).sub(1)
g: nx.Graph = nx.from_pandas_edgelist(df, source=0, target=1)

df = pd.read_csv("/kaggle/input/blogcatalog/group-edges.csv", header=None).sub(1)
nx.set_node_attributes(g, {row[0]: row[1] for row in df.iterrows()}, name="group")
vertices = tuple(sorted(g))
adj_lists = [tuple(g.neighbors(v)) for v in vertices]
V = len(g)


In [ ]:
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm

def similarity(u: Tensor, v: Tensor) -> Tensor:
    # có thể dùng khoảng cách euclid cho 1D
    # if u.dim() == 0:
    #     return u.subtract(v).abs()
    return u.dot(v)

def random_walk(v, adj_lists, walk_len):
    '''deepwalk'''
    # walk bắt đầu từ v
    walk = [v]
    for i in range(walk_len - 1):
        # chọn đỉnh kề ngẫu nhiên
        next_node = random.choice(adj_lists[v])
        # thêm đỉnh kề vào walk
        walk.append(next_node)
        # nhảy tới đỉnh kề
        v = next_node
    return walk

class DeepwalkModel(nn.Module):
    def __init__(self, V: int, emb_sz: int, leaf_pos: list) -> None:
        super(DeepwalkModel, self).__init__()
        # Hidden layer biểu diễn vector cho đỉnh
        self.embedding_layer = nn.Embedding(
            num_embeddings=V, embedding_dim=emb_sz
        )

        # Biểu diễn T bằng complete binary tree đếm từ 0
        # Luôn biểu diễn được T sao cho số lá tầng cuối = chẵn
        # <=> số nút trong = số nút lá (tức V) - 1
        self.inner_nodes_cnt = V - 1
        # Output layer (các nút trong của T) cho hierarchical softmax
        self.hsoftmax_layer = nn.Parameter(
            torch.rand((self.inner_nodes_cnt, emb_sz))
        )

        # thứ tự lá
        self.leaf_pos = leaf_pos

    def forward(self, u: int, v: int):
        """Đoán Pr(u | v_emb), trả về loss function"""
        loss = tensor(0.0)

        # Lấy ra biểu diễn vector của v
        v_emb = self.embedding_layer(tensor(v))

        # Tính nút lá ứng với u trong cây nhị phân T
        node = self.inner_nodes_cnt + self.leaf_pos[u]

        while node:
            # Kiểm tra nút hiện tại là con trái hay phải
            isLeftChild = node & 1

            # Nhảy lên nút cha
            if isLeftChild:
                node >>= 1
            else:
                node = (node - 1) >> 1

            # Lấy ra biểu diễn vector của nút trong
            node_emb = self.hsoftmax_layer[tensor(node)]

            # cập nhật loss function trên đường đi tới gốc
            if isLeftChild:
                loss -= similarity(node_emb, v_emb).sigmoid().log()
            else:
                loss -= (
                    tensor(1.0)
                    - similarity(node_emb, v_emb).sigmoid()
                ).log()
        return loss


def train_deepwalk(
    model: nn.Module,
    start_lr: float,
    end_lr: float,
    vertices: list,
    adj_lists: list[list],
    loss_records: list,
    walk_len: int,
    walks_per_vertex: int,
    window_sz: int,
    worker_threads: int,
    chunk_sz: int,
):
    lr_step = (start_lr - end_lr) / walks_per_vertex
    for i in tqdm(range(walks_per_vertex)):
        epoch_loss = 0.0
        cnt = 0
        random.shuffle(vertices)
        if worker_threads > 1:
            with mp.Pool() as pool:
                # sinh ra corpus và chia thành một batch cho mỗi walk từ một đỉnh nguồn
                batches = [
                    (
                        random_walk(v, adj_lists, walk_len),
                        window_sz,
                        start_lr,
                        model,
                    ) for v in vertices
                ]
                # huấn luyện skipgram
                results = pool.starmap(
                    func=skip_gram_deepwalk,
                    iterable=batches,
                    chunksize=chunk_sz,
                )
            for (running_loss, running_cnt) in results:
                # Ghi nhận lại hàm mất mát
                epoch_loss += running_loss
                cnt += running_cnt
        else:
            for src in vertices:
                # lấy ra đường đi ngẫu nhiên
                walk = random_walk(src, adj_lists, walk_len)
                # huấn luyện skipgram
                (running_loss, running_cnt) = skip_gram_deepwalk(
                    walk=walk,
                    window_sz=window_sz,
                    lr=start_lr,
                    model=model
                )
                # Ghi nhận lại hàm mất mát
                epoch_loss += running_loss
                cnt += running_cnt

        # tính trung bình hàm mất mát cho lần chạy
        loss_records.append(epoch_loss / cnt)

        # Giảm tuyến tính tỉ lệ học
        start_lr -= lr_step


def skip_gram_deepwalk(walk: list, window_sz: int, lr: float, model: nn.Module):
    optimizer = optim.SGD(model.parameters(), lr=lr)
    running_loss = 0.0
    cnt = 0
    # soft window
    w = random.randint(1, window_sz)
    for j, v in enumerate(walk):
        window = walk[j - w: j] + walk[j + 1: j + w + 1]
        for u in window:
            # reset gradient về 0
            optimizer.zero_grad()
            # tính hàm mất mát
            loss: Tensor = model(u, v)
            # lan truyền ngược để tính gradient cho các parameter
            loss.backward()
            # tối ưu các parameter với gradient tính đc
            optimizer.step()

            # Ghi nhận lại hàm mất mát
            running_loss += loss.item()
            cnt += 1
    return (running_loss, cnt)

### 1D Embedding

Embedding n-D bằng Deepwalk -> Embedding 1D bằng TSNE -> Sắp xếp lại thứ tự lá của Hierarchical Softmax


In [ ]:
# ánh xạ từ đỉnh -> vị trí lá
leaf_pos = list(range(V))

# embedding sẽ được chuyển về 1 chiều bằng TSNE
model = DeepwalkModel(V=V, emb_sz=EMBEDDING_SIZE, leaf_pos=leaf_pos).to(device)
loss_records = []

if WORKER_THREADS > 1:
    model.share_memory()
train_deepwalk(
    model=model,
    start_lr=START_LEARNING_RATE,
    end_lr=(START_LEARNING_RATE + END_LEARNING_RATE) / 2.0,
    vertices=list(vertices),
    adj_lists=adj_lists,
    loss_records=loss_records,
    walk_len=WALK_LENGTH,
    walks_per_vertex=PREPROCESS_WALKS_PER_VERTEX,
    window_sz=WINDOW_SIZE,
    worker_threads=WORKER_THREADS,
    chunk_sz=CHUNK_SIZE,
)

In [ ]:
plt.tick_params("y")
plt.plot(loss_records)
plt.title("Hàm mất mát theo epoch")

Chyển embedding về 1 chiều bằng TSNE rồi cập nhật lại vị trí lá cho Hierarchical Softmanx


In [ ]:
emb: Tensor = model.embedding_layer(tensor(list(g))).detach().cpu().numpy()
df = pd.DataFrame(emb)
df.to_csv("/kaggle/working/blog_prev_emb.csv", header=False, index=False)
# chuyển embedding về 1 chiều
emb = TSNE(n_components=1, perplexity=PERPLEXITY).fit_transform(emb).flatten()


# cập nhật thứ tự lá cho các đỉnh
for pos, v in enumerate(sorted(vertices, key=lambda v: emb[v])):
    leaf_pos[v] = pos

print(vertices)
print(leaf_pos)
df

### n-D Embedding


In [ ]:
model = DeepwalkModel(V=V, emb_sz=EMBEDDING_SIZE, leaf_pos=leaf_pos).to(device)
loss_records = []

if WORKER_THREADS > 1:
    model.share_memory()
train_deepwalk(
    model=model,
    start_lr=START_LEARNING_RATE,
    end_lr=END_LEARNING_RATE,
    vertices=list(vertices),
    adj_lists=adj_lists,
    loss_records=loss_records,
    walk_len=WALK_LENGTH,
    walks_per_vertex=WALKS_PER_VERTEX,
    window_sz=WINDOW_SIZE,
    worker_threads=WORKER_THREADS,
    chunk_sz=CHUNK_SIZE,
)

In [ ]:
plt.tick_params("y")
plt.plot(loss_records)
plt.title("Hàm mất mát theo epoch")

In [ ]:
emb = model.embedding_layer(tensor(list(g))).detach().cpu().numpy()
# emb = TSNE(n_components=2, perplexity=PERPLEXITY).fit_transform(emb)
# pos = {v: v_emb for v, v_emb in enumerate(emb)}

# nx.draw_networkx(g, pos)

In [ ]:
df = pd.DataFrame(emb)
df.to_csv("/kaggle/working/blog_emb.csv", header=False, index=False)
df